In [17]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [22]:
train_df = pd.read_csv('data/train.csv')
train_df['image_path'] = 'data/images/' + train_df['image_names']

test_df = pd.read_csv('data/new_labeled_test.csv')
test_df['image_path'] = 'data/images/' + test_df['image_names']

In [24]:
NUM_CLASSES = train_df['emergency_or_not'].nunique()
print(NUM_CLASSES)

2


In [25]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [28]:
train_df.head()

,image_names,emergency_or_not,image_path
0,1503.jpg,0,data/images/1503.jpg
1,1420.jpg,0,data/images/1420.jpg
2,1764.jpg,0,data/images/1764.jpg
3,1356.jpg,0,data/images/1356.jpg
4,1117.jpg,0,data/images/1117.jpg


In [31]:
test_df.head()

,image_names,emergency_or_not,Unnamed: 2,image_path
0,1960.jpg,0,NaN,data/images/1960.jpg
1,668.jpg,1,NaN,data/images/668.jpg
2,2082.jpg,0,NaN,data/images/2082.jpg
3,808.jpg,1,NaN,data/images/808.jpg
4,1907.jpg,0,NaN,data/images/1907.jpg


In [32]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='emergency_or_not',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='emergency_or_not',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='emergency_or_not',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

TypeError: If class_mode="categorical", y_col="emergency_or_not" column values must be type string, list or tuple.

In [16]:
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=Input(shape=(224,224,3)),
    classifier_activation='softmax'
)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)